In [ ]:
import json 
import os
from datetime import datetime
class DataManager:
    def __init__(self, filename="hostel_data.json"):
        self.filename = filename
        self.data = self.load_data()

    def load_data(self):
        if not os.path.exists(self.filename):
            return {"rooms": [], "students": []}
        
        try:
            with open(self.filename, 'r') as file:
                return json.load(file)
        except (json.JSONDecodeError, IOError):
            return {"rooms": [], "students": []}

    def save_data(self):
        try:
            with open(self.filename, 'w') as file:
                json.dump(self.data, file, indent=4)
        except IOError as e:
            print(f"Error saving data: {e}")

class RoomManager:
    def __init__(self, data_manager):
        self.dm = data_manager

    def add_room(self, room_num, capacity, r_type, price):
        for room in self.dm.data["rooms"]:
            if room["room_number"] == room_num:
                print(f"Error: Room {room_num} already exists!")
                return

        new_room = {
            "room_number": room_num,
            "capacity": capacity,
            "occupancy": 0,
            "type": r_type,
            "price": price
        }
        
        self.dm.data["rooms"].append(new_room)
        self.dm.save_data()
        print(f" Room {room_num} added successfully!")

    def show_availability(self):
        if not self.dm.data["rooms"]:
            print("\nNo rooms found.")
            return

        print("\n--- Room Status ---")
        print(f"{'Room':<10} {'Type':<10} {'Occupancy':<15} {'Price':<10}")
        print("-" * 50)
        for r in self.dm.data["rooms"]:
            status = f"{r['occupancy']}/{r['capacity']}"
            print(f"{r['room_number']:<10} {r['type']:<10} {status:<15} ${r['price']:<10}")
        print("-" * 50)

    def check_vacancy(self, room_num):
        for room in self.dm.data["rooms"]:
            if room["room_number"] == room_num:
                return room["occupancy"] < room["capacity"]
        return False

    def update_occupancy(self, room_num, increase=True):
        for room in self.dm.data["rooms"]:
            if room["room_number"] == room_num:
                if increase:
                    room["occupancy"] += 1
                else:
                    room["occupancy"] -= 1
                self.dm.save_data()
                return

class StudentManager:
    def __init__(self, data_manager, room_manager):
        self.dm = data_manager
        self.rm = room_manager

    def register_student(self, name, phone, room_num):
        if not self.rm.check_vacancy(room_num):
            print(f" Error: Room {room_num} is either full or does not exist.")
            return
        new_id = 1
        if self.dm.data["students"]:
            new_id = max(s["id"] for s in self.dm.data["students"]) + 1
            
        date = datetime.now().strftime("%Y-%m-%d")

        new_student = {
            "id": new_id,
            "name": name,
            "phone": phone,
            "room_number": room_num,
            "joined": date
        }

        self.dm.data["students"].append(new_student)
        self.dm.save_data()
        self.rm.update_occupancy(room_num, increase=True)
        print(f"Student {name} registered with ID {new_id}!")

    def view_student(self, student_id):
        for s in self.dm.data["students"]:
            if s["id"] == student_id:
                print("\n--- Student Details ---")
                print(f"ID: {s['id']}")
                print(f"Name: {s['name']}")
                print(f"Phone: {s['phone']}")
                print(f"Room: {s['room_number']}")
                print(f"Joined: {s['joined']}")
                return
        print("Student not found.")

    def list_all_students(self):
        """New function to list all students with IDs for Admin"""
        if not self.dm.data["students"]:
            print("\nNo students registered yet.")
            return
            
        print("\n---  All Registered Students (Admin View) ---")
        print(f"{'ID':<5} {'Name':<20} {'Room':<10}")
        print("-" * 40)
        for s in self.dm.data["students"]:
            print(f"{s['id']:<5} {s['name']:<20} {s['room_number']:<10}")
        print("-" * 40)

    def checkout_student(self, student_id):
        student_found = None
        for s in self.dm.data["students"]:
            if s["id"] == student_id:
                student_found = s
                break
        
        if student_found:
            self.dm.data["students"].remove(student_found)
            self.dm.save_data()
            self.rm.update_occupancy(student_found["room_number"], increase=False)
            print(f" Student {student_id} checked out successfully.")
        else:
            print("Student not found.")

def main():
    data_mgr = DataManager()
    rooms = RoomManager(data_mgr)
    students = StudentManager(data_mgr, rooms)

    while True:
        print("\n===  HOSTEL MANAGEMENT SYSTEM ===")
        print("1. Add New Room")
        print("2. View All Rooms")
        print("3. Register New Student")
        print("4. View Student Details (By ID)")
        print("5. Checkout Student")
        print("6. View All Students (Admin Only)")
        print("7. Exit")
        
        choice = input("Enter choice (1-7): ")

        if choice == '1':
            try:
                r_num = int(input("Enter Room Number: "))
                cap = int(input("Enter Capacity: "))
                r_type = input("Enter Type (AC/Non-AC): ")
                price = float(input("Enter Monthly Price: "))
                rooms.add_room(r_num, cap, r_type, price)
            except ValueError:
                print("Invalid input. Please enter numbers where required.")

        elif choice == '2':
            rooms.show_availability()

        elif choice == '3':
            try:
                name = input("Enter Student Name: ")
                phone = input("Enter Phone: ")
                r_num = int(input("Assign Room Number: "))
                students.register_student(name, phone, r_num)
            except ValueError:
                print("Invalid room number.")

        elif choice == '4':
            try:
                s_id = int(input("Enter Student ID: "))
                students.view_student(s_id)
            except ValueError:
                print("Invalid ID.")

        elif choice == '5':
            try:
                s_id = int(input("Enter Student ID to Checkout: "))
                students.checkout_student(s_id)
            except ValueError:
                print("Invalid ID.")

        elif choice == '6':
            password = input("Enter Admin Password: ")
            if password == "3939":
                students.list_all_students()
            else:
                print(" Incorrect Password! Access Denied.")

        elif choice == '7':
            print("Exiting system...")
            exit()
        
        else:
            print("Invalid choice, try again.")

if __name__ == "__main__":
   main()


=== 🏨 HOSTEL MANAGEMENT SYSTEM ===
1. Add New Room
2. View All Rooms
3. Register New Student
4. View Student Details (By ID)
5. Checkout Student
6. View All Students (Admin Only)
7. Exit
